In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from random import sample
from datetime import datetime

!pip install -q tensorflow_addons
import tensorflow_addons as tfa

!pip install mne
import mne

!pip install wfdb pyEDFlib PyWavelets eeglib wfdb pyeeg

%matplotlib inline
import wfdb
import pyedflib
import pywt 
import eeglib

from mne import Epochs, create_info, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.decoding import CSP
from mne.time_frequency import AverageTFR
from mne.decoding import Vectorizer
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Cargamos la lista de todos los seizures y sus tiempos en cada fichero
df_seizures = pd.read_csv('/kaggle/input/eeg-edfs-files-curated/seizures summary.csv',delimiter=';', encoding='utf8')

In [ ]:
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

lista_erroneos = ['chb10_31.edf', 'chb04_05.edf','chb04_08.edf', 'chb09_19.edf', 'chb09_06.edf', 'chb06_18.edf','chb10_20.edf',
                  'chb10_38.edf', 'chb10_12.edf', 'chb23_06.edf', 'chb06_10.edf', 'chb06_24.edf', 'chb06_09.edf', 'chb07_12.edf', 
                  'chb10_27.edf']

df_seizures = pd.read_csv('/kaggle/input/eeg-edfs-files-curated/seizures summary.csv',delimiter=';', encoding='utf8')

df_seizures = df_seizures[~df_seizures['nombre archivo'].isin(lista_erroneos)]

def lee_raw(paciente):
    
    #elegidos=[]

    #for i in np.random.choice(df_seizures['paciente'].unique(), 1):
    #for i in paciente:
        #np.random.choice(df_seizures['paciente'].unique(), 1):
        #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == i]['nombre archivo']))
    #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'],1) )
    
    #elegidos.append(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0])
    
    elegido = df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0]

    # Path with the recordings
    myPath = '/kaggle/input/eeg-edfs-files-curated/edfs/'
    # Ficheros totales (All patients)
    data = os.listdir(myPath)
    
    #epochs_all llevará todos los epochs de todos los pacientes que seleccionemos.
    epochs_all = np.empty(shape=())  # Empty array with size =  1
    data_all = np.empty(shape=()) # Empty array with size =  1
    epochs_all_10 = np.empty(shape=())  # Empty array with size =  1

    count = 0
    
    #print("Elegidos : {}".format(elegidos))
    #for i in elegidos:
        #print("type de i:{} and {}".format(type(i), i))
        #dataPath = os.path.join(myPath, data[i])
    
    dataPath = os.path.join(myPath, str(elegido))
        
    # We read our EDF recording and build a generic 'mapping' of channels that
    # we will remove since they are wrong or reference channels.
    mapping = {'EOG horizontal': 'eog', 'Resp oro-nasal': 'misc',
               'EMG submental': 'misc', 'Temp rectal': 'misc', 
               'Event marker': 'misc', '-': 'misc', '.': 'misc', 'T8-P8':'T8-P8',
               'FC1-Ref':'FC1-Ref', 'FC2-Ref':'FC2-Ref', 'FC5-Ref':'FC5-Ref',
               'FC6-Ref':'FC6-Ref', 'CP1-Ref':'CP1-Ref','CP2-Ref':'CP2-Ref',
               'CP5-Ref':'CP5-Ref', 'CP6-Ref':'CP6-Ref', 'PZ-OZ':'PZ-OZ'}
    
        # We will specifically exclude the channel with issues (T8-P8) and the weird channels (. and -)
    excluded_channels = mapping.keys()
        
    if str(elegido) in df_seizures['nombre archivo'].to_list():
        
        print("Leyendo fichero...:{}".format(elegido))
        print("Desde :{}".format(myPath+elegido))
        count +=1
        raw = mne.io.read_raw_edf(myPath+elegido, exclude=excluded_channels, preload=True, stim_channel='auto', verbose=None)
     
        raw = raw.resample(sfreq=127)

        #raw = mne.io.read_raw_edf(path, exclude=['-', 'T8-P8', '.'], verbose=False, preload=True)

        # Rename EEG channels
        ch_names = {i: i.replace('EEG ', '') for i in raw.ch_names if 'EEG' in i}
        mne.rename_channels(raw.info, ch_names)
        
        
        
        # We then apply notch filter around 60 Hz. (USA) or 50Hz (Europe) This is recommended by some papers to eliminate noise from electrodes (power line).
        raw = raw.notch_filter(freqs=[59.1, 60.9])

        # We then apply the alpha and beta filters that go from 7 to 26/28 Hz (FIRWIN type, default for MNE/Scipy)
        raw = raw.filter(l_freq=8.0, h_freq=29.0)

        # Now we mark the starts and ends of the seizures to obtain the class (Inter, Preictal, Ictal) for each second.
        seizure_init = float(df_seizures[df_seizures['nombre archivo']==elegido]['inicio seizure segundos'].to_list()[0])
        # print("seiz init", seizure_init)
        seizure_end = float(df_seizures[df_seizures['nombre archivo']==elegido]['fin seizure segundos'].to_list()[0])
        duration_seiz = float(seizure_end-seizure_init)
    
        preictal_init = float(seizure_init - 10*60)  # 10 minutos (se puede aumentar)
        preictal_duration = float(seizure_init - preictal_init)
        print("seiz_init, preictal_duration y seizure_end", seizure_init, preictal_duration, seizure_end)

        normal_antes_pre_dur = float(preictal_init)
        normal_despues_seiz_dur = float(3600 - seizure_end)

        this_file_annots = mne.Annotations(onset=[0, preictal_init, seizure_init, seizure_end ],  # in seconds
            duration=[normal_antes_pre_dur, preictal_duration, duration_seiz, normal_despues_seiz_dur],  # in seconds, too
            description=['Inter','Preictal','Ictal', 'Inter'],
            ch_names = None)
        
        raw.set_annotations(this_file_annots)
        
                
        # Recogemos los eventos de las anotaciones (para construir los futuros epochs a partir de eventos)
        # También podríamos usar duration=2.5 y overlap=0.5
        events, events_ids = mne.events_from_annotations(raw, event_id={'Preictal':0, 'Ictal':2, 'Inter':1}, 
                                                         chunk_duration=1, use_rounding=True )
        
        # Finally we crop the first minute to reduce noise (1 m. until patient settles)
        crop_file_mins = 1
        if crop_file_mins > 0:  # Cut start of file
            # Crop raw
            tmin_crop = crop_file_mins * 60
            #tmax = crop_file_mins * 60
            raw.crop(tmin=tmin_crop)
        
        # Save subject and recording information in raw.info
        # Guardamos la info del fichero: paciente/fichero.
        basename = elegido
        #print(int(basename[3:5]))

        subj_nb, rec_nb = str(basename[3:6]), str(basename[7:8])
        # print("Los integers", subj_nb, rec_nb)

        raw.info['patient_info'] = {'id': subj_nb, 'fichero_id': rec_nb}

        print('Patient_info info: {}'.format(raw.info['patient_info'])+'\n')

        if count == 1:

            raw_final = raw.get_data()

        else:

            raw_final = np.append(raw_final, raw.get_data(), axis=1)

        #mne.concatenate_epochs(epochs_list, add_offset=True, *, on_mismatch='raise', verbose=None)

        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

        # Jugando con tmin y tmax podemos obtener epochs de mas o menos duracion y/o solapamiento 
        epochs_all_tmp = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)
        
        # Construimos epochs (con el objeto Raw epochs_all_tmp de cada fichero).
        if data_all.size > 1:

            #epochs_all = np.append(epochs_all, epochs_all_tmp.get_data(), axis=0)

            epochs_PIN = epochs_all_tmp['Preictal', 'Ictal', 'Inter']
            final_df = final_df.append(epochs_PIN.to_data_frame())

            data_all_tmp = epochs_PIN.get_data()
            print('size of tmp : {}'.format(data_all_tmp.shape) +'\n')
            data_all = np.append(data_all, data_all_tmp, axis=0)

            #labels_data_tmp = epochs_all_tmp.events[:,-1]

            labels_data_all = np.append(labels_data_all, epochs_PIN.events[:,-1], axis = 0)

            #epochs_all_10 = np.append(epochs_all_10, epochs_all_tmp_10, axis=0)

        else:
            
            epochs_all = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)

            epochs_PIN = epochs_all['Preictal', 'Ictal', 'Inter']
                
            # Para 'unificar' clases por tamanos. 
            # epochs_PIN.equalize_event_counts(list(events_ids.keys()))
                
            final_df = epochs_PIN.to_data_frame()

            data_all = epochs_PIN.get_data()

            #print('size of dataall :{}'.format(data_all.shape) + '\n')
            labels_data_all = epochs_PIN.events[:,-1]

            #epochs_all_10 = epochs_all_tmp_10

#    else:

#        continue
    final_df['cat'] = [1 if i == 'Preictal' else 2 if i == 'Ictal' else 0 for i in final_df['condition']]
    
    return final_df

In [ ]:
def compute_cwt(DF, wavelet, ventana=10, solapamiento=0, fs = 128):
    
    fs = fs
    final_df = DF
    waveletname = pywt.ContinuousWavelet(wavelet)
    escalas = pywt.scale2frequency(waveletname.name, np.arange(8,30.5,2)) / (1/fs)
    print("Numero de escalas que vamos a probar : {}".format(len(escalas)))
    ventana = ventana
    incremento = fs * ventana  # La ventana me da el incremento o 'avance' en cada iteracion
    solapamiento = int(solapamiento * fs)
    
    train_size = int(np.floor(final_df.shape[0]/(ventana*fs)))
    
    n_channels = len(final_df.columns) - 4
    # Redondeamos para evitar el último epoch que desbordaría
    tamano_resultado_train = int(round(train_size,0))
    
    # Matriz resultado en train (tamaño muestras, escalas, freq.samp, canales)
    train_data_cwt = np.ndarray(shape=(tamano_resultado_train, len(escalas), fs, n_channels))
    train_data_cwt_labels = np.ndarray(shape=(tamano_resultado_train, 1, n_channels))
    
    for ii in range(0,tamano_resultado_train):
        if ii == int(np.floor(tamano_resultado_train/2)):
            print("CWT en set training a la mitad...{}".format(ii*incremento))
    
        for jj in range(0,n_channels):
        
            start = final_df['epoch'].min()
 
            signal = final_df.iloc[ii*(incremento - solapamiento):(ii+1)*(incremento-solapamiento), jj+3:jj+4]
    
            # Calculamos la clase con el 'techo' del valor de la mediana de cada ventana
            clase_intervalo = int(np.ceil(np.median(final_df.iloc[ii*(incremento- solapamiento) : (ii+1)*(incremento- solapamiento), -1])))
  
            train_data_cwt_labels[ii, : ,jj] = clase_intervalo
        
            coeff, freq = pywt.cwt(signal, escalas, waveletname, 1)
        
            coeff_approx = coeff[:,:fs]
        
            train_data_cwt[ii, :, :, jj] = abs(coeff_approx[:,:,-1])
    
    print("Final CWT decomp. \n") 
    
    return train_data_cwt, train_data_cwt_labels

In [ ]:
fs = 128
escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,2) / (1/fs))
print("Numero de escalas que vamos a probar : {}".format(len(escalas)))

In [ ]:
# Resultados individualizados

segundos = [30]
wavelets = ['mexh']
solapamientos = [0, 1]
resultados = dict()
#for i in df_seizures['paciente'].unique():
for i in np.random.choice(df_seizures['paciente'].unique(),1).tolist():
    temp = lee_raw(i)
    #lectura_edf = obtain_edf(i)
    for j in segundos:
        for k in wavelets:        
            #resultados.update({'paciente_'+i+'_'+str(j)+'_'+k: temp})
            for l in solapamientos:
                resultados.update({'paciente_'+i+'_'+str(j)+'_'+k+'solap_'+str(l): compute_cwt(temp, k, j, l)})

In [ ]:
from tensorflow import keras
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.models import Sequential
from keras.callbacks import History 
#history = History()

num_escalas = list(resultados.values())[0][0].shape[1]
coeficientes = list(resultados.values())[0][0].shape[2]
canales = list(resultados.values())[0][0].shape[3]
input_shape = (num_escalas, coeficientes , canales)

num_classes = 3
batch_size = 16
epochs = 50

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy',
                tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
                tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')])

In [ ]:
    METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
def create_callbacks(model, metric = METRICS):
    
    #guardar_path = '/home/inaki/temp/mejor_modelo_'+ model.name + '.h5'
    
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(
    #    filepath = guardar_path,
    #    monitor = metric,
    #    mode ='max',
    #    save_best_only=True,
    #    verbose=1,
    #)

    reduccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [reduccionlr, earlystop]         
    
    return callbacks

In [ ]:
def process_model(data, labels):
    from random import sample
    
    train_data_cwt = data
    train_data_cwt_labels = labels
    
    #print(train_data_cwt.shape)
    #print(train_data_cwt_labels.shape)
    
    items = [item for sublist in train_data_cwt_labels[:,:,-1] for item in sublist]

    import collections

    # using Counter from Dict() to find frequency of elements
    balanceo = collections.Counter(items)
    suma = sum(balanceo.values())
    for k, v in balanceo.items():
        balanceo[k] = (1/v)*(suma/3)
    print("Balanceo clases : {}".format(dict(balanceo)))

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    #num_instances, escalas, coef, num_canales = train_data_cwt.shape
    train_data = np.reshape(train_data_cwt, (-1, train_data_cwt.shape[-1]))
    train_data = scaler.fit_transform(train_data)
    train_data = np.reshape(train_data, (train_data_cwt.shape[0], train_data_cwt.shape[1], train_data_cwt.shape[2], train_data_cwt.shape[-1]))
    
    random_num_generated = int(sample(range(0,99), 1)[0])
    
    # Ensure that we get at least one sample of each class on both train and test
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
    y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    # Ensure that we get at least one sample of each class on both train and test
    while len(set(np.argmax(y_test_cat, axis=1).tolist())) < len(set(np.argmax(y_train_cat, axis=1).tolist())):
        random_num_generated = int(sample(range(0,99), 1)[0])
        X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
        y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
        y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    batch_size = 16
    epochs = 50
    
    METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics= METRICS)
    
    history = History()
    history = model.fit(X_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_cat),
          #callbacks = create_callbacks(model),
          callbacks = [history],
          class_weight = balanceo)

    ultimos = dict()
    
    for k, v in history.history.items():
        ultimos.update({k: v[-1]})  
    
    return ultimos

Operamos ahora con los resultados de todos los CWTs obtenidos por paciente, ejecutando el modelo CNN para cada uno de los pacientes y guardando los resultados.

In [ ]:
historias = dict()
for k, v in resultados.items():
    #a = v[0]
    #b = v[1]
    #print(a.shape, b.shape)
    hist_temp = process_model(v[0], v[1])
    historias.update({k:hist_temp})

In [ ]:
#historias_df.head()
#for k, v in resultados.items():
#    print(k, v[0].shape[1], v[0].shape[2], v[0].shape[3])
#        #v[0][0].shape[1],v[0][0].shape[2],v[0][0].shape[3])

In [ ]:
from datetime import datetime
historias_df = pd.DataFrame(historias)
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
print(dt_string)
historias_df.to_csv('/kaggle/working/historias_df'+dt_string+'.csv', sep=',', na_rep='', header=True, index=True, mode='w')

In [ ]:
from IPython.display import HTML
import base64

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = f'<a target="_blank">{title}</a>'
    return HTML(html)


#historias_df.to_csv('/kaggle/working/historias_df'+dt_string+'.csv', sep=',', na_rep='', header=True, index=True, mode='w')

In [ ]:
#create_download_link(historias_df, filename = '/kaggle/working/historias_df'+dt_string+'.csv')

In [ ]:
mode = pywt.Modes.smooth

def dwt_signal_decomp(data, w):
    """Decompose and plot a signal S.
    S = An + Dn + Dn-1 + ... + D1
    """
    w = pywt.Wavelet(w)
    a = data
    ca = []
    cd = []
    for i in range(5):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)

    rec_a = []
    rec_d = []

    for i, coeff in enumerate(ca):
        coeff_list = [coeff, None] + [None] * i
        rec_a.append(pywt.waverec(coeff_list, w))

    for i, coeff in enumerate(cd):
        coeff_list = [None, coeff] + [None] * i
        rec_d.append(pywt.waverec(coeff_list, w))
        
    return ca, cd